<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#test

In [ ]:
!wget -O NGIDS-DS.rar https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download

--2025-04-20 15:09:26--  https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download
Resolving unsworks.unsw.edu.au (unsworks.unsw.edu.au)... 54.253.215.118
Connecting to unsworks.unsw.edu.au (unsworks.unsw.edu.au)|54.253.215.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content [following]
--2025-04-20 15:09:28--  https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content
Reusing existing connection to unsworks.unsw.edu.au:443.
HTTP request sent, awaiting response... 200 200
Length: unspecified [application/x-rar-compressed]
Saving to: ‘NGIDS-DS.rar’

NGIDS-DS.rar            [             <=>    ] 941.51M  15.8MB/s    in 61s     

2025-04-20 15:10:30 (15.3 MB/s) - ‘NGIDS-DS.rar’ saved [987249484]



In [ ]:
!unrar x /content/NGIDS-DS.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/NGIDS-DS.rar

Creating    NGIDS                                                     OK
Creating    NGIDS/NGIDS-DS-v1                                         OK
Extracting  NGIDS/NGIDS-DS-v1/feature_descr.csv                            0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/ground_truth.csv                             0%  OK 
Creating    NGIDS/NGIDS-DS-v1/host logs                               OK
Extracting  NGIDS/NGIDS-DS-v1/host logs/1.csv                              0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/10.csv                             0%  1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/11.csv                             1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/12.csv                             1%  2%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/13.csv                             2%  OK 
Extrac

In [ ]:
!rm -rf /content/NGIDS

#class

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import math
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
class Preprocessing():
  def __init__(self):
    pass

  def __file_path_collection(self, folder_path):
    try:
      joined_list = []
      for file_name in os.listdir(folder_path):
        file_name = os.path.join(folder_path, file_name)
        if os.path.isfile(file_name) and file_name.endswith('.csv'):
          joined_list.append(file_name)
      return joined_list

    except Exception as e:
      return f"An error occurred: {e}"


  def rename_columns(self, folder_path):
    files = self.__file_path_collection(folder_path)
    try:
      for file in tqdm(files, desc="Renaming columns") :
        df = pd.read_csv(file)
        column_name = df.columns.tolist()
        df.rename(columns={
            column_name[0]:'Date',
            column_name[1]:'Time',
            column_name[2]:'Unique_Identification',
            column_name[3]:'Execution_Path',
            column_name[4]:'System_Calls_Identifiers',
            column_name[5]:"Event's_Unique_Identification",
            column_name[6]:'attacks',
            column_name[7]:"Sub_Type_Attack",
            column_name[8]:"Label"
            }, inplace=True)
        df.to_csv(file, index=False)
      return f"done files..."

    except Exception as e:
      return f"An error occurred: {e}"


  def data_encoding(self, folder_path, column_number : int):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    all_value = []
    try:
      for file in tqdm(files, desc="Extract all values from files"):
        df = pd.read_csv(file)
        all_value.extend(df.iloc[:, column_number].dropna().unique())
        #print( len(df.iloc[:, column_number].dropna().unique()) , file )
      encoder.fit(all_value)

      for file in tqdm(files, desc="encoder data"):
        df = pd.read_csv(file)
        df.iloc[:, column_number] = encoder.transform(df.iloc[:, column_number])
        #print(len(df.iloc[:, column_number].dropna().unique()), df.iloc[:, column_number].dropna().unique(), file )
        df.to_csv(file, index=False)
      return f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"


  def paths_encoding(self, folder_path, column_number: int):
    files = self.__file_path_collection(folder_path)
    all_unique_value = []
    unique_value = []
    try:
      for file in tqdm(files, desc="read all files"):
        df = pd.read_csv(file)
        unique_value.extend(df.iloc[:,column_number].unique())
        #df["Tokenized"] = unique_value.apply(lambda p: p.strip("/").split("/"))
        for tokens in unique_value:
          all_unique_value.extend(tokens)

      print( len(all_unique_value), all_unique_value)

    except Exception as e:
      return f"An error occurred: {e}"


  def file_collection(self, folder_path):
    try:
      file_paths = self.__file_path_collection(folder_path)
      if file_paths:
        output_folder = os.path.join(os.getcwd(),'files')
        os.makedirs(output_folder ,exist_ok=True)
        output_file = os.path.join(output_folder,"final.csv")

        for file in file_paths:

          first = True
          chunk = pd.read_csv(file, chunksize=10000)
          for part in chunk:
            part.to_csv(output_file, mode='a', header=first, index=False)
            first = False
          print(f"done file: {file}")

    except Exception as e:
      print(f"An error occurred: {e}")


a = Preprocessing()
#a.file_path_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.file_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.rename_columns( '/content/NGIDS/NGIDS-DS-v1/host logs')
a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=6)
a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=7)

#a.paths_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=3)

encoder data: 100%|██████████| 99/99 [10:03<00:00,  6.10s/it]


'done file...'

#test

In [ ]:

from google.colab import files

files.download('/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp /content/NGIDS/NGIDS-DS-v1/NGIDS.pcap /content/drive/MyDrive/1000$

In [ ]:

from scapy.all import rdpcap
from collections import Counter

# تحميل الملف
packets = rdpcap("/mnt/data/NGIDS.pcap")

# عدد الحزم
total_packets = len(packets)

# إحصائيات البروتوكولات والمصادر والوجهات
protocols = Counter()
sources = Counter()
destinations = Counter()
packet_summaries = []

for pkt in packets:
    # حفظ البروتوكول
    proto = pkt.name
    protocols[proto] += 1

    # حفظ المصدر والوجهة إن توفرا
    if pkt.haslayer("IP"):
        sources[pkt["IP"].src] += 1
        destinations[pkt["IP"].dst] += 1

    # حفظ ملخص لكل حزمة (أول 10 فقط للتقرير)
    if len(packet_summaries) < 10:
        packet_summaries.append(pkt.summary())

# تجهيز النتائج
{
    "total_packets": total_packets,
    "top_protocols": protocols.most_common(5),
    "top_sources": sources.most_common(5),
    "top_destinations": destinations.most_common(5),
    "sample_packets": packet_summaries,
}

In [ ]:
unique_attacks = list(set([item for sub in c for item in sub]))
unique_attacks

['Exploits',
 'Generic',
 'Shellcode',
 'Worms',
 'Reconnaissance',
 'Denial of Service',
 'Backdoors',
 'normal']

In [ ]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.6 MB/s eta 0:00:00


In [17]:

from scapy.all import rdpcap

# تحميل الحزم من الملف
packets = rdpcap("/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap")

# طباعة معلومات أول 5 حزم
for pkt in packets[:5]:
    print(pkt.summary())

Ether / IP / TCP 59.166.0.0:60294 > 10.40.85.31:80 S
Ether / IP / TCP 59.166.0.3:22311 > 10.40.85.31:1024 S
Ether / IP / TCP 59.166.0.4:63239 > 10.40.85.31:27322 S
Ether / IP / TCP 59.166.0.2:37241 > 10.40.85.31:23962 S
Ether / IP / TCP 59.166.0.5:17808 > 10.40.85.31:80 S


In [18]:

from scapy.all import rdpcap

# استبدل المسار بمسار ملفك
pcap_file = "/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap"
packets = rdpcap(pcap_file)

for i, pkt in enumerate(packets, 1):
    print(f"========== Packet {i} ==========")
    print(pkt.show(dump=True))  # يعرض جميع طبقات وبروتوكولات الحزمة
    print("\n--- Raw Data (Hexdump) ---")
    hexdump(pkt)

KeyboardInterrupt: 

In [ ]:

from scapy.all import rdpcap, hexdump

# مسار ملف pcap
pcap_file = "/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap"

# قراءة جميع الحزم
packets = rdpcap(pcap_file)

# استعراض شامل لكل حزمة
for i, pkt in enumerate(packets, 1):
    print(f"\n========== Packet {i} ==========")

    # طباعة جميع الطبقات والحقول
    print(pkt.show(dump=True))

    # طباعة البيانات الخام (raw payload) كـ hex
    print("\n--- Raw Data (Hexdump) ---")
    hexdump(pkt)

    print("=" * 60)

Streaming output truncated to the last 5000 lines.
     chksum    = 0x8847
     src       = 175.45.176.3
     dst       = 10.40.85.32
     \options   \
###[ TCP ]###
        sport     = 39293
        dport     = 80
        seq       = 2690965287
        ack       = 0
        dataofs   = 7
        reserved  = 0
        flags     = R
        window    = 16383
        chksum    = 0x4f38
        urgptr    = 0
        options   = [('MSS', 9158), ('WScale', 0), ('NOP', None)]


--- Raw Data (Hexdump) ---
0000  EC B1 D7 54 60 BC 02 1A C5 00 00 00 08 00 45 00  ...T`.........E.
0010  00 30 75 07 00 00 FF 06 88 47 AF 2D B0 03 0A 28  .0u......G.-...(
0020  55 20 99 7D 00 50 A0 64 DF 27 00 00 00 00 70 04  U .}.P.d.'....p.
0030  3F FF 4F 38 00 00 02 04 23 C6 03 03 00 01        ?.O8....#.....

========== Packet 19611 ==========
###[ Ethernet ]###
  dst       = ec:b1:d7:54:60:bc
  src       = 02:1a:c5:00:00:00
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0


In [ ]:
df = pd.read_csv("/content/NGIDS/NGIDS-DS-v1/ground_truth.csv")
df

,11/03/2016,9:21:36,Backdoors,All Batch,Cisco Network Registrar Default Credentials Backdoor Access (https://strikecenter.bpointsys.com/bps/strikes/backdoors/cve_2011_2024_cisco_network_registrar_auth_bypass.xml),CVE 2011-2024 (http://cve.mitre.org/cgi-bin/cvename.cgi?name=2011%2d2024)BID 48076 (http://www.securityfocus.com/bid/48076)OSVDB 72720 (http://www.osvdb.org/72720)CVSS-Critical (https://strikecenter.bpointsys.com/bps/reference/CVSS/10.0%20%28AV%3aN%2fAC%3aL%2fAu%3aN%2fC%3aC%2fI%3aC%2fA%3aC%29),IP 175.45.176.0:0->10.40.85.32:0 175.45.176.0:60689->10.40.85.32:8080
0,14/03/2016,12:14:24,Backdoors,All Batch,BlackEnergy Botnet Command and Control Communi...,http://atlas-public.ec2.arbor.net/docs/BlackEn...,175.45.176.1:3495->10.40.85.32:58782
1,15/03/2016,4:19:12,Backdoors,All Batch,Backdoor: Cisco Prime LAN Management (https://...,CVE 2012-6392 (http://cve.mitre.org/cgi-bin/cv...,175.45.176.0:13177->10.40.85.32:514
2,14/03/2016,9:36:00,Backdoors,All Batch,phpmyadmin 3.5.2.2 Backdoor Access and Code Ex...,CVE 2012-5159 (http://cve.mitre.org/cgi-bin/cv...,IP 175.45.176.2:0->10.40.85.32:0 175.45.176.2:...
3,15/03/2016,11:45:36,Backdoors,All Batch,Android AndroidKungFu Malware Command and Cont...,http://about-threats.trendmicro.com/malware.as...,175.45.176.3:61508->10.40.85.32:7500
4,11/03/2016,9:21:36,Backdoors,All Batch,Cisco Network Registrar Default Credentials Ba...,CVE 2011-2024 (http://cve.mitre.org/cgi-bin/cv...,IP 175.45.176.0:0->10.40.85.32:0 175.45.176.0:...
...,...,...,...,...,...,...,...
313920,11/03/2016,9:07:12,Worms,All Batch,Lupper.A XML-RPC Propogation Request Variant 1...,CVE 2005-1921 (http://cve.mitre.org/cgi-bin/cv...,175.45.176.3:58526->10.40.85.32:80
313921,11/03/2016,5:16:48,Worms,All Batch,Microsoft SQL Server Slammer/Saphire Worm (htt...,MSB MS02-039 (http://www.microsoft.com/technet...,UDP 175.45.176.3:50917->10.40.85.32:1434
313922,11/03/2016,7:12:00,Worms,All Batch,Lupper.A XML-RPC Propogation Request Variant 1...,CVE 2005-1921 (http://cve.mitre.org/cgi-bin/cv...,175.45.176.3:36155->10.40.85.32:80
313923,11/03/2016,4:48:00,Worms,All Batch,Lupper.A XML-RPC Propogation Request Variant 1...,CVE 2005-1921 (http://cve.mitre.org/cgi-bin/cv...,175.45.176.1:20900->10.40.85.32:80


In [ ]:
df = pd.read_csv("/content/NGIDS/NGIDS-DS-v1/host logs/9.csv")
df['Sub_Type_Attack'].value_counts()
#df

,count
Sub_Type_Attack,
52,610823
44,2378
27,928
24,776
2,452
10,412
30,400
41,330
40,329
